In [12]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
#from skimage import io
from sklearn.preprocessing import OneHotEncoder
import pickle


In [13]:
df=pd.read_csv('../Data/1M_unique_processed_data.csv')

In [14]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset='track_uri',inplace=True)
df.shape

(66243, 31)

In [15]:
# List of columns to drop
columns_to_drop = ['name', 'num_holdouts', 'pid', 'num_tracks', 'num_samples', 'pos', 'time_signature','genres']

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)
# Dropping rows with missing values in specified columns


In [16]:
df = df.sample(frac=0.1, random_state=42)  # Adjust fraction size as needed

# Display the shape of the sampled dataset
print(df.shape)

(6624, 23)


In [17]:
df.dropna(subset=['danceability'], inplace=True)


In [18]:
df.columns

Index(['artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri',
       'track_duration_ms', 'album_name', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'type', 'audio_duration_ms',
       'release_date', 'popularity', 'artist_pop'],
      dtype='object')

In [19]:
df.isnull().sum()

artist_name            0
track_uri              0
artist_uri             0
track_name             0
album_uri              0
track_duration_ms      0
album_name             0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
type                   0
audio_duration_ms      0
release_date          23
popularity            23
artist_pop           101
dtype: int64

In [20]:
xgb_model = pickle.load(open('../Models/xgb.pkl', 'rb'))  # Update with your file path
scaler = pickle.load(open('../Models/scaler.pkl', 'rb'))
label_encoder = pickle.load(open('../Models/encoder.pkl', 'rb'))


In [21]:


# Keep only relevant features
input_features = df[['danceability', 'energy', 'key', 'loudness', 'mode',
                            'speechiness', 'acousticness', 'instrumentalness',
                            'liveness', 'valence', 'tempo']]

# Scale the features using the previously saved scaler
scaled_features = scaler.transform(input_features)
scaled_df = pd.DataFrame(scaled_features, columns=input_features.columns)

# Make predictions using the XGBoost model
predictions = xgb_model.predict(scaled_df)

# Inverse transform the numeric labels back to genre names
predicted_genres = label_encoder.inverse_transform(predictions)

# Add the predicted 'genre' column to the original dataframe
df['genre'] = predicted_genres




In [22]:
df.columns

Index(['artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri',
       'track_duration_ms', 'album_name', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'type', 'audio_duration_ms',
       'release_date', 'popularity', 'artist_pop', 'genre'],
      dtype='object')

In [23]:
columns_to_keep = ['artist_name', 'track_name', 'artist_pop', 'popularity','release_date','genre','track_uri','artist_uri']
df = df.dropna(subset=columns_to_keep)

# Selecting only the columns of interest
df = df[columns_to_keep]
df.shape

(5024, 8)

In [24]:
df.to_csv('../Data/Spotify_with_genre.csv', index=False)
